In [1]:
library(broom)
library(nnet)
library(forcats)
library(stargazer)
library(sjPlot)
library(data.table)
library(lme4)
library(lmerTest)


Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 

Warning message:
“package ‘sjPlot’ was built under R version 3.6.3”Install package "strengejacke" from GitHub (`devtools::install_github("strengejacke/strengejacke")`) to load all sj-packages at once!
Loading required package: Matrix
Warning message:
“package ‘Matrix’ was built under R version 3.6.2”Warning message:
“package ‘lmerTest’ was built under R version 3.6.2”
Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step



In [1]:
#data = read.table('/shared/2/projects/framing/results/full_datasheet_11-13-20.tsv',header=TRUE,sep='\t')
data = read.table('/shared/2/projects/framing/results/data_5_percent_11-13-20.tsv',header=TRUE,sep='\t')


In [3]:
data$year[data$year>2019] <- 2019
head(data)

X,Capacity.and.Resources,Crime.and.Punishment,Cultural.Identity,Economic,Episodic,External.Regulation.and.Reputation,Fairness.and.Equality,Health.and.Safety,Hero,...,log_chars,log_favorites,log_followers,log_following,log_retweets,log_statuses,month,op_ideology,opposed_ideology,year
0,0,0,1,1,1,0,0,0,1,...,4.812184,0.0000000,10.063436,8.933796,1.098612,11.308371,10,NA,NA,2018
1,0,0,0,0,0,0,0,0,0,...,4.727388,NA,10.340161,10.343547,NA,12.727886,6,NA,NA,2018
2,1,0,0,1,0,0,0,0,0,...,5.525453,0.6931472,7.075809,6.989335,1.098612,8.806724,9,NA,NA,2018
3,0,0,1,0,0,0,0,0,0,...,5.713733,0.0000000,3.178054,4.043051,0.000000,5.902633,12,NA,NA,2018
4,0,0,0,0,0,1,0,0,0,...,5.669881,0.0000000,3.496508,4.025352,0.000000,8.589514,12,NA,NA,2018
5,0,1,1,0,1,0,1,0,0,...,5.429346,0.0000000,6.922644,7.317876,0.000000,9.809671,12,NA,NA,2018


In [2]:
colnames(data)

[1] "X"                                        
 [2] "Unnamed..0"                               
 [3] "Capacity.and.Resources"                   
 [4] "Crime.and.Punishment"                     
 [5] "Cultural.Identity"                        
 [6] "Economic"                                 
 [7] "Episodic"                                 
 [8] "External.Regulation.and.Reputation"       
 [9] "Fairness.and.Equality"                    
[10] "Health.and.Safety"                        
[11] "Hero"                                     
[12] "Hero..Cultural.Diversity"                 
[13] "Hero..Integration"                        
[14] "Hero..Worker"                             
[15] "Legality..Constitutionality..Jurisdiction"
[16] "Morality.and.Ethics"                      
[17] "Policy.Prescription.and.Evaluation"       
[18] "Political.Factors.and.Implications"       
[19] "Public.Sentiment"                         
[20] "Quality.of.Life"                          
[21] "Security.and.Defense"                     
[22] "Thematic"                                 
[23] "Threat"                                   
[24] "Threat..Fiscal"                           
[25] "Threat..Jobs"                             
[26] "Threat..National.Cohesion"                
[27] "Threat..Public.Order"                     
[28] "Victim"                                   
[29] "Victim..Discrimination"                   
[30] "Victim..Global.Economy"                   
[31] "Victim..Humanitarian"                     
[32] "Victim..War"                              
[33] "country"                                  
[34] "date"                                     
[35] "has_hashtag"                              
[36] "has_mention"                              
[37] "has_url"                                  
[38] "id_str"                                   
[39] "ideology"                                 
[40] "is_quote_status"                          
[41] "is_reply"                                 
[42] "is_verified"                              
[43] "log_chars"                                
[44] "log_favorites"                            
[45] "log_followers"                            
[46] "log_following"                            
[47] "log_retweets"                             
[48] "log_statuses"                             
[49] "month"                                    
[50] "op_ideology"                              
[51] "opposed_ideology"                         
[52] "year"

In [5]:
cols <- cbind(colnames(data))
data$id_str <- NULL
data$X <- NULL
data$Unnamed..0 <- NULL
data$op_ideology <- NULL
data$opposed_ideology <- NULL
data$log_favorites <- NULL
data$log_retweets <- NULL
frames <- cols[1:30]
features <- cols[31:length(cols)]
factor_features <- cbind('country','has_hashtag','has_mention','has_url','is_quote_status','is_reply',
                         'is_verified','date','month','year')
data[,frames] <-lapply(data[,frames],as.factor)
data[,factor_features] <-lapply(data[,factor_features],as.factor)

In [6]:
frames

[1] "Capacity.and.Resources"                   
 [2] "Crime.and.Punishment"                     
 [3] "Cultural.Identity"                        
 [4] "Economic"                                 
 [5] "Episodic"                                 
 [6] "External.Regulation.and.Reputation"       
 [7] "Fairness.and.Equality"                    
 [8] "Health.and.Safety"                        
 [9] "Hero"                                     
[10] "Hero..Cultural.Diversity"                 
[11] "Hero..Integration"                        
[12] "Hero..Worker"                             
[13] "Legality..Constitutionality..Jurisdiction"
[14] "Morality.and.Ethics"                      
[15] "Policy.Prescription.and.Evaluation"       
[16] "Political.Factors.and.Implications"       
[17] "Public.Sentiment"                         
[18] "Quality.of.Life"                          
[19] "Security.and.Defense"                     
[20] "Thematic"                                 
[21] "Threat"                                   
[22] "Threat..Fiscal"                           
[23] "Threat..Jobs"                             
[24] "Threat..National.Cohesion"                
[25] "Threat..Public.Order"                     
[26] "Victim"                                   
[27] "Victim..Discrimination"                   
[28] "Victim..Global.Economy"                   
[29] "Victim..Humanitarian"                     
[30] "Victim..War"

In [7]:
country_features <- c('country','date','month','year',
                      'has_hashtag','has_mention','has_url',
                     'is_quote_status','is_reply','is_verified',
                     'log_chars','log_followers','log_following','log_statuses')

ideology_features <- c('country','date','month','year',
                      'has_hashtag','has_mention','has_url',
                     'is_quote_status','is_reply','is_verified',
                     'log_chars','log_followers','log_following','log_statuses',
                      'ideology')

In [9]:
# Analysis 1: Predict Frames from Country (logistic regression)
all_country_models_full <- list()
full_results = data.frame()
for (i in 1:length(frames)) {
    df <- subset(data, select = c(frames[i],country_features))
    colnames(df)[1] <- 'y'
    df$country <- relevel(df$country, ref = 'US')
    model <- glmer(y ~ . + 
                   (1 | year/month/date) 
                   - date - month - year
                   ,data = df,family='binomial',
                  nAGQ=0L,
                  control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5),calc.derivs = FALSE))
   all_country_models_full[[i]] <- model
   res <- tidy(model)
   res$frame <- frames[i]
   full_results <- rbind(full_results,res)
   stargazer(model,no.space=TRUE,single.row=TRUE,title=frames[i])
  }
#write.table(full_results, "/shared/2/projects/framing/results/frame_building_country/predict_frame_from_country_full_11-18.tsv",sep='\t',)



% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Wed, Nov 18, 2020 - 04:54:04 PM
\begin{table}[!htbp] \centering 
  \caption{Capacity.and.Resources} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & y \\ 
\hline \\[-1.8ex] 
 countryEU & $-$0.091$^{***}$ (0.015) \\ 
  countryGB & 0.403$^{***}$ (0.007) \\ 
  has\_hashtag1 & $-$0.301$^{***}$ (0.011) \\ 
  has\_mention1 & 0.057$^{***}$ (0.009) \\ 
  has\_url1 & 0.001 (0.011) \\ 
  is\_quote\_status1 & 0.324$^{***}$ (0.009) \\ 
  is\_reply1 & 0.359$^{***}$ (0.009) \\ 
  is\_verified1 & $-$0.239$^{***}$ (0.020) \\ 
  log\_chars & 0.730$^{***}$ (0.008) \\ 
  log\_followers & $-$0.019$^{***}$ (0.003) \\ 
  log\_following & 0.006$^{**}$ (0.003) \\ 
  log\_statuses & $-$0.005$^{**}$ (0.002) \\ 
  Constant & $-$6.801$^{***}$ (0.082) \\ 
 \hline \\[-1

In [10]:
# Analysis 2: Predict Frames from Ideology (logistic regression)
all_ideology_models_full <- list()
full_results = data.frame()
for (i in 1:length(frames)) {
    df <- subset(data, select = c(frames[i],ideology_features))
    df <- df[df$country=='US', ]
    df$country <- NULL
    colnames(df)[1] <- "y"
    model <- glmer(y ~ . + 
                   (1 | year/month/date) 
                   - date - month - year
                   ,data = df,family='binomial',
                  nAGQ=0L,
                  control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5),calc.derivs = FALSE))
   all_ideology_models_full[[i]] <- model
   res <- tidy(model)
   res$frame <- frames[i]
   full_results <- rbind(full_results,res)
   stargazer(model,no.space=TRUE,single.row=TRUE,title=frames[i])
  }
write.table(full_results, "/shared/2/projects/framing/results/frame_building_ideology/predict_frame_from_ideology_full_11-18.tsv",sep='\t',)


% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Thu, Nov 19, 2020 - 02:03:02 PM
\begin{table}[!htbp] \centering 
  \caption{Capacity.and.Resources} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & y \\ 
\hline \\[-1.8ex] 
 has\_hashtag1 & $-$0.268$^{***}$ (0.014) \\ 
  has\_mention1 & $-$0.056$^{***}$ (0.011) \\ 
  has\_url1 & $-$0.049$^{***}$ (0.013) \\ 
  is\_quote\_status1 & 0.248$^{***}$ (0.012) \\ 
  is\_reply1 & 0.329$^{***}$ (0.011) \\ 
  is\_verified1 & $-$0.084$^{***}$ (0.025) \\ 
  log\_chars & 0.767$^{***}$ (0.010) \\ 
  log\_followers & $-$0.013$^{***}$ (0.004) \\ 
  log\_following & $-$0.019$^{***}$ (0.004) \\ 
  log\_statuses & $-$0.023$^{***}$ (0.003) \\ 
  ideology & 0.178$^{***}$ (0.002) \\ 
  Constant & $-$6.763$^{***}$ (0.101) \\ 
 \hline \\[-1.8ex] 
Observations & 1,6

In [ ]:
# Analysis 3: Predict Favorites from Frames (Linear Regression)
fav_data <- data
fav_data$log_retweets <- NULL
fav_model = lmer(log_favorites ~ . + 
                 (1 | year/month/date) + 
                 (1|country) 
                 - date - month - year - country - ideology - op_ideology - opposed_ideology
                  , data = fav_data)

In [ ]:
# Analysis 4: Predict Retweets from Frames (Linear Regression)
rt_data <- data
rt_data$log_favorites <- NULL
rt_model = lmer(log_retweets ~ . + 
                 (1 | year/month/date) + 
                 (1|country)
                 - date - month - year - country - ideology - op_ideology - opposed_ideology, data = rt_data)

In [ ]:
class(rt_model) <- "lmerMod"
class(fav_model) <- "lmerMod"


out_file = '/shared/2/projects/framing/results/favorite_and_retweet_11-13.txt'
stargazer(fav_model,rt_model,no.space=TRUE,single.row=TRUE,type='text',report=('vcs*p'),out=out_file)

In [ ]:
country_results = read.table("/shared/2/projects/framing/results/frame_building_country/predict_frame_from_country_11-14.tsv",sep='\t')
ideology_results = read.table("/shared/2/projects/framing/results/frame_building_ideology/predict_frame_from_ideology_11-14.tsv",sep='\t')

country_orig_p <- country_results$p.value
ideology_orig_p <- ideology_results$p.value

country_corrected_p <- p.adjust(country_orig_p, method = "holm", n = length(country_orig_p))
ideology_corrected_p <- p.adjust(ideology_orig_p, method = "holm", n = length(ideology_orig_p))

country_results['p.corrected'] = country_corrected_p
ideology_results['p.corrected'] = ideology_corrected_p

In [ ]:
results_path <- "/shared/2/projects/framing/results/frame_building_"
write.table(country_results,paste(results_path,'country/predict_frame_from_country_11-14_corrected_p.tsv',sep=''),sep='\t')
write.table(ideology_results,paste(results_path,'ideology/predict_frame_from_ideology_11-14_corrected_p.tsv',sep=''),sep='\t',)




In [14]:
length(all_ideology_models_full)

[1] 30

In [15]:
#save(model, file="model.Rdata")
all_ideology_models_full

[[1]]
Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 0) [glmerMod]
 Family: binomial  ( logit )
Formula: y ~ . + (1 | year/month/date) - date - month - year
   Data: df
      AIC       BIC    logLik  deviance  df.resid 
 747638.9  747823.9 -373804.5  747608.9   1680149 
Random effects:
 Groups            Name        Std.Dev.
 date:(month:year) (Intercept) 0.26817 
 month:year        (Intercept) 0.24963 
 year              (Intercept) 0.09826 
Number of obs: 1680164, groups:  
date:(month:year), 615; month:year, 23; year, 2
Fixed Effects:
     (Intercept)      has_hashtag1      has_mention1          has_url1  
        -6.76262          -0.26768          -0.05551          -0.04869  
is_quote_status1         is_reply1      is_verified1         log_chars  
         0.24767           0.32945          -0.08420           0.76671  
   log_followers     log_following      log_statuses          ideology  
        -0.01335          -0.01949  

In [21]:
for (i in 1:length(frames)) {
    print(i)
    frame = frames[i]
    model = all_ideology_models_full[i]
    filename = paste('/shared/2/projects/framing/results/frame_building_ideology/model_ideology_',frame,'.Rdata',sep='')
    save(model, file=filename)

}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30


In [22]:
for (i in 1:length(frames)) {
    print(i)
    frame = frames[i]
    model = all_country_models_full[i]
    filename = paste('/shared/2/projects/framing/results/frame_building_country/model_country_',frame,'.Rdata',sep='')
    save(model, file=filename)

}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30


In [25]:
ideology_results = read.table("/shared/2/projects/framing/results/frame_building_ideology/predict_frame_from_ideology_full_11-18.tsv",sep='\t')
country_results = read.table("/shared/2/projects/framing/results/frame_building_country/predict_frame_from_country_full_11-18.tsv",sep='\t')

In [27]:
country_orig_p <- country_results$p.value
ideology_orig_p <- ideology_results$p.value

country_corrected_p <- p.adjust(country_orig_p, method = "holm", n = length(country_orig_p))
ideology_corrected_p <- p.adjust(ideology_orig_p, method = "holm", n = length(ideology_orig_p))

country_results['p.corrected'] = country_corrected_p
ideology_results['p.corrected'] = ideology_corrected_p

In [28]:
results_path <- "/shared/2/projects/framing/results/frame_building_"
write.table(country_results,paste(results_path,'country/predict_frame_from_country_11-19_corrected_p.tsv',sep=''),sep='\t')
write.table(ideology_results,paste(results_path,'ideology/predict_frame_from_ideology_11-19_corrected_p.tsv',sep=''),sep='\t',)




In [ ]:
for (i in 1:length(frames)) {
    frame = frames[i]
    model = all_country_models_full[i]
    frame_corrected_p <- country_results[country_results$frame == frame, ]$p.corrected
    class(model) <- "lmerMod"
    stargazer(model,no.space=TRUE,single.row=TRUE,type='latex',
          p.auto=FALSE,p=(c(frame_corrected_p)),star.cutoffs=c(.05,.01,.005))
    

}

In [45]:
print(load('/shared/2/projects/framing/results/frame_building_ideology/model_ideology_Economic.Rdata'))

[1] "model"


In [55]:
stargazer(model[[1]],no.space=TRUE,single.row=TRUE,type='text')


                        Dependent variable:    
                    ---------------------------
                                 y             
-----------------------------------------------
has_hashtag1             -0.140*** (0.007)     
has_mention1              -0.003 (0.006)       
has_url1                 -0.067*** (0.008)     
is_quote_status1         0.181*** (0.006)      
is_reply1                0.138*** (0.006)      
is_verified1             -0.043*** (0.013)     
log_chars                0.973*** (0.006)      
log_followers            -0.069*** (0.002)     
log_following            0.025*** (0.002)      
log_statuses             0.041*** (0.002)      
ideology                 0.048*** (0.001)      
Constant                 -6.375*** (0.066)     
-----------------------------------------------
Observations                 1,680,164         
Log Likelihood             -882,709.100        
Akaike Inf. Crit.          1,765,448.000       
Bayesian Inf. Crit.        1,765,633.00

In [47]:
model

[[1]]
Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 0) [glmerMod]
 Family: binomial  ( logit )
Formula: y ~ . + (1 | year/month/date) - date - month - year
   Data: df
      AIC       BIC    logLik  deviance  df.resid 
1765448.3 1765633.3 -882709.1 1765418.3   1680149 
Random effects:
 Groups            Name        Std.Dev.
 date:(month:year) (Intercept) 0.24093 
 month:year        (Intercept) 0.14303 
 year              (Intercept) 0.07103 
Number of obs: 1680164, groups:  
date:(month:year), 615; month:year, 23; year, 2
Fixed Effects:
     (Intercept)      has_hashtag1      has_mention1          has_url1  
       -6.374950         -0.139576         -0.002763         -0.066871  
is_quote_status1         is_reply1      is_verified1         log_chars  
        0.180808          0.137690         -0.043021          0.973042  
   log_followers     log_following      log_statuses          ideology  
       -0.068615          0.024868  